In [1]:
import seaborn as sns
import pandas as pd

In [2]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df = df.drop(columns=['deck', 'embark_town', 'alive']) #deck null value banyak, embarktown diwakili embarked dan alive sebagai target  di survived

df.isnull().sum()

survived        0
pclass          0
sex             0
age           177
sibsp           0
parch           0
fare            0
embarked        2
class           0
who             0
adult_male      0
alone           0
dtype: int64

In [4]:
df = df.dropna(subset='embarked')
df.isnull().sum()

survived        0
pclass          0
sex             0
age           177
sibsp           0
parch           0
fare            0
embarked        0
class           0
who             0
adult_male      0
alone           0
dtype: int64

In [5]:
X = df.drop(columns=['survived'])
y = df['survived']

In [6]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

In [7]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alone
828,3,male,NaN,0,0,7.7500,Q,Third,man,True,True
238,2,male,19.0,0,0,10.5000,S,Second,man,True,True
453,1,male,49.0,1,0,89.1042,C,First,man,True,False
516,2,female,34.0,0,0,10.5000,S,Second,woman,False,True
383,1,female,35.0,1,0,52.0000,S,First,woman,False,False


In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 666 entries, 828 to 712
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   pclass      666 non-null    int64   
 1   sex         666 non-null    object  
 2   age         531 non-null    float64 
 3   sibsp       666 non-null    int64   
 4   parch       666 non-null    int64   
 5   fare        666 non-null    float64 
 6   embarked    666 non-null    object  
 7   class       666 non-null    category
 8   who         666 non-null    object  
 9   adult_male  666 non-null    bool    
 10  alone       666 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(3), object(3)
memory usage: 48.9+ KB


In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

imp_scale = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('Scaler', RobustScaler())
])

transformer = ColumnTransformer([
    ('imputer_mean', imp_scale, ['age']),
    ('robust', RobustScaler(), ['fare']),
    ('onehot', OneHotEncoder(drop='first'), ['sex', 'embarked', 'class', 'who', 'adult_male', 'alone'])
], remainder='passthrough')

transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputer_mean',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('Scaler', RobustScaler())]),
                                 ['age']),
                                ('robust', RobustScaler(), ['fare']),
                                ('onehot', OneHotEncoder(drop='first'),
                                 ['sex', 'embarked', 'class', 'who',
                                  'adult_male', 'alone'])])

In [10]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score

resample = SMOTE(random_state=42)
model = RandomForestClassifier(random_state=42)

estimator = Pipeline([
    ('transorm', transformer),
    ('resample', resample),
    ('model', model),
])
estimator.fit(X_train, y_train)
y_pred = estimator.predict(X_test)
accuracy_score(y_test, y_pred)

0.7668161434977578

In [11]:
import pickle

model = pickle.dump(estimator, open('data/model.sav', 'wb'))